In [68]:
from bertopic import BERTopic
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from umap import UMAP
from hdbscan import HDBSCAN
import hdbscan
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import PartOfSpeech
import os
import json
import openml

In [69]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
import sys
from pathlib import Path

project_root = os.getenv('PROJECT_ROOT')
sys.path.append(str(project_root))
print(project_root)
print(os.getcwd())

/Users/ivang/PycharmProjects/pythonProject/
/Users/ivang/PycharmProjects/pythonProject/notebooks


# Data preparation

In [ ]:
# datasets = openml.datasets.list_datasets()
# print(openml.datasets.list_datasets()[531])

# ids = list(datasets.keys())
# print("Number of datasets: ", len(ids))

# # Removing non-dataset entries 
# ids.remove(4537)
# ids.remove(4546)
# ids.remove(4562)

# # Fetching dataset
# datasets = openml.datasets.get_datasets(ids, download_data=False, download_qualities=False)
# datasets[0]

In [ ]:
# df = pd.DataFrame([dataset.__dict__ for dataset in datasets])
# # Add the OpenML URL to the dataframe
# df['openml_url'] = [dataset.openml_url for dataset in datasets]
# df['upload_date'] = [dataset.upload_date for dataset in datasets]
# # All datasets which have None description, make it an empty string ""
# df['description'] = df['description'].fillna('')
# df

Save dataframe, so datasets aren't refetched every time.

In [ ]:
# # save dataframe, so datasets aren't refetched every time, save everything as is, None don't get turned to NaN, etc.
# df.to_pickle("data/datasets.pkl")

Load datasets from the saved CSV

In [ ]:
df = pd.read_pickle("data/datasets.pkl")
df

,ignore_attribute,dataset_id,name,version,description,cache_format,format,creator,contributor,collection_date,...,parquet_file,_dataset,_parquet_url,_features,_qualities,_no_qualities_found,data_pickle_file,data_feather_file,feather_attribute_file,openml_url
0,None,2,anneal,1,**Author**: Unknown. Donated by David Sterling...,pickle,ARFF,"[David Sterling, Wray Buntine]",David Sterling and Wray Buntine,1990,...,None,None,http://openml1.win.tue.nl/dataset2/dataset_2.pq,"{0: [0 - family (nominal)], 1: [1 - product-ty...",None,False,None,None,None,https://www.openml.org/d/2
1,None,3,kr-vs-kp,1,Author: Alen Shapiro\nSource: [UCI](https://ar...,pickle,ARFF,Alen Shapiro,Rob Holte,1989-08-01,...,None,None,http://openml1.win.tue.nl/dataset3/dataset_3.pq,"{0: [0 - bkblk (nominal)], 1: [1 - bknwy (nomi...",None,False,None,None,None,https://www.openml.org/d/3
2,None,4,labor,1,**Author**: Unknown\n**Source**: Collective Ba...,pickle,ARFF,Collective Bargaining Review of Labour Canada,Stan Matwin,1988-11-01,...,None,None,http://openml1.win.tue.nl/dataset4/dataset_4.pq,"{0: [0 - duration (numeric)], 1: [1 - wage-inc...",None,False,None,None,None,https://www.openml.org/d/4
3,None,5,arrhythmia,1,"**Author**: H. Altay Guvenir, Burak Acar, Hald...",pickle,ARFF,"[H. Altay Guvenir, Burak Acar, Haldun Muderris...",None,1998-01-01,...,None,None,http://openml1.win.tue.nl/dataset5/dataset_5.pq,"{0: [0 - age (numeric)], 1: [1 - sex (nominal)...",None,False,None,None,None,https://www.openml.org/d/5
4,None,6,letter,1,**Author**: David J. Slate \n**Source**: [UCI...,pickle,ARFF,David J. Slate,None,1991-01-01,...,None,None,http://openml1.win.tue.nl/dataset6/dataset_6.pq,"{0: [0 - x-box (numeric)], 1: [1 - y-box (nume...",None,False,None,None,None,https://www.openml.org/d/6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5674,None,46204,US-Births,1,Number of births in the United States.\n\nFrom...,pickle,arff,"Randall Pruim, Daniel Kaplan, Nicholas Horton",Bruno Belucci,1968 - 1988,...,None,None,https://openml1.win.tue.nl/datasets/0004/46204...,"{0: [0 - id_series (nominal)], 1: [1 - date (s...",None,False,None,None,None,https://www.openml.org/d/46204
5675,None,46205,Solar-Power,1,Solar power production in MW recorded per ever...,pickle,arff,"Godahewa, Rakshitha; Bergmeir, Christoph; Web...",None,2020,...,None,None,https://openml1.win.tue.nl/datasets/0004/46205...,"{0: [0 - id_series (nominal)], 1: [1 - value_0...",None,False,None,None,None,https://www.openml.org/d/46205
5676,None,46206,Wind-Power,1,Wind power production in MW recorded per every...,pickle,arff,"Godahewa, Rakshitha; Bergmeir, Christoph; Web...",None,2020,...,None,None,https://openml1.win.tue.nl/datasets/0004/46206...,"{0: [0 - id_series (nominal)], 1: [1 - value_0...",None,False,None,None,None,https://www.openml.org/d/46206
5677,None,46207,Wind-Power,2,Wind power production in MW recorded per every...,pickle,arff,"Godahewa, Rakshitha; Bergmeir, Christoph; Web...",None,2020,...,None,None,https://openml1.win.tue.nl/datasets/0004/46207...,"{0: [0 - id_series (nominal)], 1: [1 - value_0...",None,False,None,None,None,https://www.openml.org/d/46207


In [ ]:
# import importlib

# from src.augmenters import tag_augmenter, feature_augmenter, dataset_augmenter, similarity_augmenter, name_augmenter, \
#     scrapy_augmenter, llm_prompt_augmenter

# importlib.reload(tag_augmenter)
from src.augmenters import TagAugmenter

# importlib.reload(feature_augmenter)
from src.augmenters import FeatureAugmenter

# importlib.reload(dataset_augmenter)
from src.augmenters import DatasetAugmenter

# importlib.reload(similarity_augmenter)
from src.augmenters import SimilarityAugmenter

# importlib.reload(name_augmenter)
from src.augmenters import NameAugmenter

# importlib.reload(scrapy_augmenter)
from src.augmenters import ScrapyAugmenter

# importlib.reload(llm_prompt_augmenter)
from src.augmenters import LLMPromptAugmenter

description_column = 'description'
dataset_id_column = 'dataset_id'
tag_column = 'tag'
features_column = '_features'
name_column = 'name'
augmented_column = 'augmented_description'
similar_datasets_column = 'similar_datasets'
scraped_column = 'scraped_data'
prompt_description_column = 'prompt_description'

augmenters = [
    TagAugmenter(description_column, tag_column, augmented_column, dataset_id_column, json_file_path=os.path.join(project_root, 'src', 'augmenters', 'GPT_semantic_tags.json')),
    NameAugmenter(description_column, name_column, augmented_column),
    FeatureAugmenter(description_column, features_column, augmented_column, max_features=100, reduce_features=True),
    ScrapyAugmenter(description_column, scraped_column, augmented_column),
    SimilarityAugmenter(description_column, augmented_column, similar_datasets_column),
    # Only load LLMPromptAugmenter if the datasets have been augmented with the LLM below in the code
    # LLMPromptAugmenter(prompt_description_column)
]

dataset_augmenter = DatasetAugmenter(augmenters=augmenters)
augmented_df = dataset_augmenter.augment(df.copy())
augmented_df

### Save augmented descriptions

In [ ]:
if not os.path.exists("data"):
    os.makedirs("data")

# augmented_df[[dataset_id_column, augmented_column]].to_csv("data/augmented_descriptions.csv", index=False)
# Only save the prompt descriptions if the datasets have been augmented with the LLM below in the code
augmented_df[prompt_description_column].to_csv("data/prompt_descriptions.csv", index=False)

# data = augmented_df[augmented_column].tolist()
# Only load the prompt descriptions if the datasets have been augmented with the LLM below in the code
data = augmented_df[prompt_description_column].tolist()

### Use LLM to rewrite the augmented descriptions

This step is run on demand and not every single time, as it requires a powerful GPU to run the LLM model. The model is used to rewrite the augmented descriptions to make them more human-readable.

In [ ]:
# # Only run once
# from llama_cpp import Llama

# llm = Llama(model_path="./mixtral-8x7b-instruct-v0.1.Q5_K_M.gguf", n_ctx=16384, n_threads=10,
# llm = Llama(model_path="./models/Meta-Llama-3-70B-Instruct.Q4_K_M.gguf", n_ctx=8092, n_threads=10,
#             n_gpu_layers=70)

In [ ]:
# # Only run once
# from openai import OpenAI
# client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
# # Only run once
# import anthropic

# client = anthropic.Anthropic(api_key=os.getenv("CLAUDE_API_KEY"))

In [ ]:
# # Only run once
# def get_prompt(datum):
#     return f"""{datum}
# # Rewrite the text above fixing formatting errors, such as dashes, underscores, etc. in words (instead make the words human readable). If the features are human unreadable and you can't fix that, skip the features that are unreadable. Remove author names and names of people, but keep dataset name and tags. Remove numbers. Stick to the same content that's already provided in the text, and just give the text, no additional text before or after. Be as thorough and verbose as possible in reproducing the original text. If the text is too short, try to expand it by making the sentences longer with the main themes. Also, extract a few (3-10) concise and human-readable keywords from the text at the end. Don't say "Here is the rewritten text" or anything like that, don't explain yourself, show don't tell, just give the final result!!!""" 

In [ ]:
# # Only run once
# Local LLM
# for i in tqdm(range(0, len(data))):
#     output = llm(
#         f"""{get_prompt(data[i])""",
#         max_tokens=None,
#         echo=True,
#         stop=['<|eot_id|>'],
#         temperature=0.7,
#         top_p=0.95, top_k=40, 
#         repetition_penalty=1.1
#     )
#     # output_text = output["choices"][0]["text"].split("original text.")[1]
#     # output_text
#     with open(f'data/dataset_{i}.txt', 'w') as f:
#         f.write(str(output))

# Chat-GPT 3.5
# for i in tqdm(range(0, len(data)):
#     stream = client.chat.completions.create(
#         model="gpt-3.5-turbo",
#         messages=[{"role": "user", "content": f'''{data[i]}'''}],
#         stream=True,
#         temperature=0.3
#     )
#     output_text = ''
#     for chunk in stream:
#         output_text += chunk.choices[0].delta.content or ""
#         with open(f'data/dataset_openai{i}.txt', 'w') as f:
#             f.write(str(output_text))

# Claude 3.5
# for i in tqdm(range(0, len(data))):
#     message = client.messages.create(
#         model="claude-3-5-sonnet-20240620",
#         max_tokens=1000,
#         temperature=0,
#         messages=[
#             {
#                 "role": "user",
#                 "content": [
#                     {
#                         "type": "text",
#                         "text": f'''get_prompt(data[i])'''
#                     }
#                 ]
#             }
#         ]
#     )
#     with open(f'data/dataset_claude{i}.txt', 'w') as f:
#         f.write(str(message.content[0].text))

# Calculate embeddings

In [80]:
# # Only run once
# import voyageai
# vo = voyageai.Client(api_key=os.getenv('VOYAGE_AI_API_URL'))

# batch_size = 128
# embeddings = []

# # Use tqdm to show a progress bar
# for i in tqdm(range(0, len(data), batch_size), desc="Embedding Batches"):
#     # Embed the batch of data
#     batch_embeddings = vo.embed(
#         data[i : i + batch_size],
#         model="voyage-3",
#         input_type="document",
#     ).embeddings
    
#     # Append the embeddings to the list
#     embeddings.append(batch_embeddings)

# # Concatenate the embeddings into a single array
# embeddings = np.concatenate(embeddings, axis=0)
# np.save("embeddings.npy", embeddings)

Embedding Batches: 100%|██████████| 16/16 [00:43<00:00,  2.74s/it]


In [82]:
embeddings = np.load("embeddings.npy")

In [83]:
embeddings.shape

(2000, 1024)

# Train topic model
Dimensionality reduction and clustering are most important, there is also a CountVectorizer, c-TF-IDF calculation and PartOfSpeech, but they matter less

In [84]:
# Only run once
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1, 2))
# Run below if this cell complains about not finding the model
# !python -m spacy download en_core_web_lg
representation_model = PartOfSpeech(model="en_core_web_lg")

umap_model = UMAP(
    n_neighbors=15,
    n_components=5,
    min_dist=0.0,
)

hdbscan_model = HDBSCAN(
    min_cluster_size=15,
    metric="euclidean",
    cluster_selection_method="eom",
    prediction_data=True,
)

ctfidf_model = ClassTfidfTransformer()

model = BERTopic(
    verbose=True,
    min_topic_size=2,
    ctfidf_model=ctfidf_model,
    calculate_probabilities=True,
    umap_model=umap_model,
    # embedding_model=embedding_model, # don't pass embedding model, just pass embeddings in the next step (pre-computed)
    vectorizer_model=vectorizer_model,
    representation_model=representation_model,
)

topics, probs = model.fit_transform(data, embeddings)
np.save('topics.npy', topics)
np.save('probs.npy', probs)

2024-10-06 01:24:46,185 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-10-06 01:24:49,292 - BERTopic - Dimensionality - Completed ✓
2024-10-06 01:24:49,293 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-10-06 01:24:51,682 - BERTopic - Cluster - Completed ✓
2024-10-06 01:24:51,684 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-10-06 01:25:26,791 - BERTopic - Representation - Completed ✓


In [85]:
# model.save("model", serialization="safetensors", save_ctfidf=True)

In [ ]:
model = BERTopic.load("model")

In [ ]:
topics = np.load('topics.npy')
probs = np.load('probs.npy')

In [87]:
prompt = '''Based on the below information, extract and synthesize human-readable tags/keywords/themes from the text, capitalized first letters of words. What is the main human-readable theme or subject matter discussed in the provided texts? What is the overarching, high-level theme of the texts, e.g. "Music", "Sports", "Environment", etc.? Please provide overarching themes that tie the different pieces of information together. What is/are the overarching, highest level theme(s) that you could use as a keyword(s)? Prefer single word tags/keywords, e.g. "Tennis" rather than "Tennis Match", "Prison" rather than "Prison Time", etc.! Some examples of human-readable themes are   "Agriculture", "Astronomy", "Chemistry", "Computational Universe", "Computer Systems", "Climate and Environment", "Culture", "Demographics", "Earth Science", "Economics", "Education", "Engineering", "Finance", "Geography", "Government", "Games", "Health", "History", "Human Activities", "Images", "Language", "Law", "Life Science", "Machine Learning", "Manufacturing", "Mathematics", "Medicine", "Meteorology", "Physical Sciences", "Politics", "Social Media", "Sociology", "Statistics", "Text & Literature",  "Transportation". Also, don't give very similar tags/keywords, e.g. "Wine" and "Red Wine", just give one or the other in these cases. Avoid tags/keywords that are too specific, e.g. "Serine Threonine Protein Kinase". Good theme examples are: "Birds", "Species Migration", "Air Pollution", or "War", "Government", "International Relations", "Politics". Another important rule to obey - place more focus on the dataset names for theme extraction. And be concise in theme generation, e.g. instead of "Income Prediction", say "Income", instead of "Demographic Information", say "Demographics"! Also, extract the theme of the text, what it is about, instead of the type of problem it is, for instance we don't care about "Regression", "Numerical Features", "Data Analysis", "Data", "Outliers", "Subsampling" or things of that sort, but we care about the ESSENCE of the text! Say {"Themes": [...], "Overarching themes": [...]} and give your answer in JSON format.
For example, for this text:
Text 1: The Biden administration is preparing to roll out a sweeping border executive action as early as Tuesday, according to two sources familiar with the discussions, who cautioned that timing is fluid.

White House officials have begun reaching out to mayors who represent cities along the US southern border to potentially join President Joe Biden when he announces the order, two other sources familiar with those conversations said.

For weeks, administration officials have been working through an executive action that would dramatically limit migrants’ ability to seek asylum at the US southern border — part of a strategy to try to give Biden the upper hand on one of his Republican rival’s key campaign issues. The action is designed to potentially blunt Republican attacks on border security and preempt former President Donald Trump ahead of the first presidential debate, which will be held on June 27 on CNN.
---
Text 2: Now that a New York jury has convicted former President Donald Trump of all 34 felony charges of falsifying business records, the next obvious question is: Can a convicted felon run for president?

Definitely.

Trump meets all three requirements. There is, arguably, another criterion laid out in the 14th Amendment, where it states that no one who has previously taken an oath of office who engages in insurrection can be an officer of the US. But the US Supreme Court ruled earlier this year that Congress would have to pass a special law invoking this prohibition. That’s not happening any time soon.

Judge Juan Merchan has scheduled Trump’s sentencing for July 11, which happens to be four days before the start of the Republican National Convention that is scheduled to take place in Milwaukee.
a
It is technically possible, although perhaps unlikely for a first-time offender, that Trump could be sentenced to prison time.
---
This would be your answer:
{"Themes": ["Biden Administration", "Border", "Executive Action", "Asylum", "Immigration", "Trump", "Felony", "Business Records", "Presidential Campaign", "Republican", "Debate", "Former President", "Conviction", "Sentencing", "Prison", "14th Amendment", "Insurrection", "Supreme Court", "Republican National Convention"], "Overarching themes": ["Politics", "Government", "Law", "Justice", "Elections"]}
---
Now, the above was just an example. Now, do it for the following text(s), be concise!:
[DOCUMENTS]
The topic is described by the following keywords: [KEYWORDS]
---
Remember, extract and synthesize human-readable tags/keywords/themes from the text, capitalized first letters of words. What is the main human-readable theme or subject matter discussed in the provided texts? What is the overarching, high-level theme of the texts, e.g. "Music", "Sports", "Environment", etc.? Please provide overarching themes that tie the different pieces of information together. What is/are the overarching, highest level theme(s) that you could use as a keyword(s)? Prefer single word tags/keywords, e.g. "Tennis" rather than "Tennis Match", "Prison" rather than "Prison Time", etc.! Some examples of human-readable themes are   "Agriculture", "Astronomy", "Chemistry", "Computational Universe", "Computer Systems", "Climate and Environment", "Culture", "Demographics", "Earth Science", "Economics", "Education", "Engineering", "Finance", "Geography", "Government", "Games", "Health", "History", "Human Activities", "Images", "Language", "Law", "Life Science", "Machine Learning", "Manufacturing", "Mathematics", "Medicine", "Meteorology", "Physical Sciences", "Politics", "Social Media", "Sociology", "Statistics", "Text & Literature",  "Transportation". Also, don't give very similar tags/keywords, e.g. "Wine" and "Red Wine", just give one or the other in these cases. Avoid tags/keywords that are too specific, e.g. "Serine Threonine Protein Kinase". Good theme examples are: "Birds", "Species Migration", "Air Pollution", or "War", "Government", "International Relations", "Politics". Another important rule to obey - place more focus on the dataset names for theme extraction. And be concise in theme generation, e.g. instead of "Income Prediction", say "Income", instead of "Demographic Information", say "Demographics"! Also, extract the theme of the text, what it is about, instead of the type of problem it is, for instance we don't care about "Regression", "Numerical Features", "Data Analysis", "Data", "Outliers", "Subsampling" or things of that sort, but we care about the ESSENCE of the text! Say {"Themes": [...], "Overarching themes": [...]} and give your answer in JSON format.
'''

In [88]:
# Only run once
new_topics = model.reduce_outliers(data, topics, probabilities=probs, strategy="probabilities")
model.update_topics(data, topics=new_topics)

2024-10-06 01:26:50,709 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


In [89]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,45,0_shots_team_period_game,"[shots, team, period, game, goal, he, play, ho...",[NHL PLAYOFF RESULTS FOR 4/20/93.\n\n---------...
1,1,39,1_car_cars_v6_ford,"[car, cars, v6, ford, engine, mustang, probe, ...",[an excellent automatic can be found in the su...
2,2,28,2_gun_militia_guns_firearms,"[gun, militia, guns, firearms, knife, weapons,...",[/ iftccu:talk.politics.guns / mikey@ccwf.cc.u...
3,3,25,3_cryptography_submissions_systems_organization,"[cryptography, submissions, systems, organizat...",[In article <1993Apr2.050451.7866@ucsu.Colorad...
4,4,24,4_voltage_noise_output_ohm,"[voltage, noise, output, ohm, digital, rms, am...","[MAMISHEV, ALEXANDER VALENTINO (avm1993@sigma...."
...,...,...,...,...,...
245,245,6,245_revelation_senses_you_truth,"[revelation, senses, you, truth, me, highway, ...",[[reply to timmbake@mcl.ucsb.edu (Bake Timmons...
246,246,4,246_dam9543_intersection_dirtbike_duck,"[dam9543, intersection, dirtbike, duck, hawk, ...",[In article <csundh30.735598088@ursa> csundh30...
247,247,8,247_nodes_unix_advance_thanks,"[nodes, unix, advance, thanks, ken, macintosh,...","[Hi netlanders,\n\nDoes anybody know if there ..."
248,248,4,248_tobacco_health_smokeless_coli,"[tobacco, health, smokeless, coli, o157h7, amo...",[In article <1qk6v3INNrm6@lynx.unm.edu> bhjell...


In [90]:
# Only run once
# from extensions import MistralAI
from src.bertopic_extensions import MistralAI
from mistralai import Mistral

client = Mistral(api_key=os.getenv("MISTRAL_API_KEY"))

llm = MistralAI(
    client,
    prompt=prompt,
    model="mistral-large-latest",
    nr_docs=100,
    max_tokens = 8192,
    cache=False,
    generator_kwargs={
        "response_format": {
            "type": "json_object"
        },
    },
)

model.update_topics(data, topics=new_topics, representation_model=llm)

In [ ]:
# model.save("model_fine_tuned", serialization="safetensors", save_ctfidf=True)

In [35]:
# # Only run this step if the topic_representations aren't updated for some reason (error, etc.)
# import ast
# import json

# with open("../src/bertopic_extensions/_cache/mistral.json") as f:
#     mistral = json.load(f)

# for i, topic_data in enumerate(mistral):
#     themes = [(theme, 1) for theme in topic_data['Themes']]
#     overarching_themes = [(theme, 2) for theme in topic_data['Overarching themes']]
#     model.topic_representations_[i] = themes + overarching_themes

# Zeroshot step

In [ ]:
# Only run once
top_k = 2
top_k_topics = []
for i in range(len(data)):
    top_k_indices = np.argsort(probs[i])[::-1][:top_k]
    top_k_indices = [index for index in top_k_indices]
    top_k_indices = [index for index in top_k_indices if index != -1]
    top_k_topics.append([str(topic) for topic in top_k_indices])

model.get_topic_info(int(top_k_topics[0][0]))

In [ ]:
# Only run once
import json

with open('model_fine_tuned/topics.json') as f:
    topics_json = json.load(f)

topics_json

In [99]:
# Only run once
def split_themes(data):
    themes_dict = {}
    overarching_themes_dict = {}
    count = 0
    for topic, representations in data['topic_representations'].items():
        themes = []
        overarching_themes = []

        for rep in representations:
            themes.extend([theme for theme in json.loads(rep[0])['Themes']])
            overarching_themes.extend([theme for theme in json.loads(rep[0])['Overarching themes']])
        
        themes = list(dict.fromkeys(themes))
        overarching_themes = list(dict.fromkeys(overarching_themes))
        
        themes_dict[topic] = ', '.join(themes)
        overarching_themes_dict[topic] = ', '.join(overarching_themes)
        count+=1
    
    return themes_dict, overarching_themes_dict

themes_dict, overarching_themes_dict = split_themes(topics_json)

print("Themes Dictionary:")
for k, v in themes_dict.items():
    print(f"{k}: {v}")

print("\nOverarching Themes Dictionary:")
for k, v in overarching_themes_dict.items():
    print(f"{k}: {v}")

Themes Dictionary:
0: Hockey, Teams, Players, Games, Goals, Playoffs, Penalties, Shots, Periods, Season, Stanley Cup, Fans, Trades, Injuries, Captains, Coaches, Standings, Polls
1: Car, Cars, V6, Ford, Engine, Mustang, Probe, GT, V8, Gear, Auto, Nissan, Automatic, Airbag, Automobile, Maxima, Autos, Rotor, Opel, Strut, Manta, Buy, Manual, Safety
2: Guns, Militia, Firearms, Weapons, Nuclear, Government, Law, Politics, Crime, Safety, Constitution, Rights, Legislation
3: Cryptography, Privacy, Security, Encryption, Government, Policy, Data, Freedom, Rights, Organization, NSA, EFF, ACLU, NRA, TEMPEST, RSA, PGP, Export, Law, Constitution, Assembly, Arms, Symposium, Submissions, Systems, Files, Keys
4: Electronics, Circuits, Power Supply, Voltage, Current, Digital, Analog, Signal Processing, Measurement, Amplifiers, Components, Electrical Engineering, Data Acquisition, Sound, Noise
5: Clipper, Encryption, Algorithm, Government, Chip, Privacy, EXPN, NSA, Proposal, Sternlight, Denning, Csrcncsl

In [132]:
# Only run once

def append_to_json(file_path, new_data):
    """Append new data to a JSON array stored in a file."""
    if os.path.exists(file_path):
        with open(file_path, 'r+') as file:
            # Load existing data
            try:
                data = json.load(file)
            except json.JSONDecodeError:
                data = []
            # Append new data
            data.append(new_data)
            # Move the file pointer to the beginning
            file.seek(0)
            # Write the updated data
            json.dump(data, file, indent=2)
            # Truncate the file to the current size
            file.truncate()
    else:
        with open(file_path, 'w') as file:
            # Write new data in a list
            json.dump([new_data], file, indent=2)
            
import requests

API_URL = os.getenv('HUGGINGFACE_DEDICATED_API_URL')
headers = {
	"Accept" : "application/json",
	"Authorization": f"Bearer {os.getenv('HUGGINGFACE_DEDICATED_API_KEY')}",
	"Content-Type": "application/json" 
}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

hypothesis_template = "This text is about: {}"

for i, topics in tqdm(enumerate(top_k_topics), total=len(top_k_topics), desc="Processing Topics"):
    topic_labels_list = []
    overarching_labels_list = []
    
    for topic_index in topics:
        # Using themes from the themes_dict
        labels = themes_dict[str(topic_index)]
        labels = labels.split(", ")
        topic_labels_list.append(labels)

        # Using overarching themes from the overarching_themes_dict
        overarching_labels = overarching_themes_dict[str(topic_index)]
        overarching_labels = overarching_labels.split(", ")
        overarching_labels_list.append(overarching_labels)
    
    # Flatten the list of lists and remove duplicates
    topic_list = [item for sublist in topic_labels_list for item in sublist]
    topic_list = list(set(topic_list))
    overarching_list = [item for sublist in overarching_labels_list for item in sublist]
    overarching_list = list(set(overarching_list))
    
    # Query the API instead of zeroshot_model
    output = query({
        "inputs": data[i],  # Assuming `data[i]` is the input text for the zeroshot classifier
        "parameters": {
            "candidate_labels": topic_list,
            "hypothesis_template": hypothesis_template,
            "multi_label": True
        }
    })
    
    append_to_json('tags.json', output)

    # Query for overarching labels
    output_overarching = query({
        "inputs": data[i],
        "parameters": {
            "candidate_labels": overarching_list,
            "hypothesis_template": hypothesis_template,
            "multi_label": True
        }
    })
    
    append_to_json('overarching_tags.json', output_overarching)
    
    # Print labels with scores > 0.7
    for label, score in zip(output['labels'], output['scores']):
        if score > 0.7:
            print(f"Label: {label}, Score: {score}")
    
    # Print overarching labels with scores > 0.5
    for label, score in zip(output_overarching['labels'], output_overarching['scores']):
        if score > 0.5:
            print(f"Overarching Label: {label}, Score: {score}")

Processing Topics:   0%|          | 1/2000 [00:05<3:10:57,  5.73s/it]

Label: Science, Score: 0.9980769753456116
Label: Evolution, Score: 0.995388925075531
Label: Theory, Score: 0.9721987843513489
Label: Question, Score: 0.9169695973396301
Label: Existence, Score: 0.7879037857055664
Overarching Label: Science, Score: 0.9980769753456116


Processing Topics:   0%|          | 2/2000 [00:12<3:33:42,  6.42s/it]

Label: Comics, Score: 0.997971773147583
Label: Marvel, Score: 0.9422758221626282
Overarching Label: Comics, Score: 0.997971773147583
Overarching Label: Entertainment, Score: 0.9957425594329834
Overarching Label: Literature, Score: 0.8633022308349609


Processing Topics:   0%|          | 3/2000 [00:18<3:20:14,  6.02s/it]

...

Overarching Label: Business, Score: 0.7615827322006226


Processing Topics: 100%|█████████▉| 1999/2000 [2:18:44<00:04,  4.38s/it]

Label: Guns, Score: 0.9988630414009094
Label: Firearms, Score: 0.9985544085502625
Label: Weapons, Score: 0.9978575706481934
Label: Politics, Score: 0.980798065662384
Label: Law, Score: 0.9606895446777344
Label: Rights, Score: 0.9552730321884155
Label: Crime, Score: 0.9025211930274963
Label: Legislation, Score: 0.8702861666679382
Overarching Label: Guns, Score: 0.9988630414009094
Overarching Label: Weapons, Score: 0.9978575706481934
Overarching Label: Politics, Score: 0.980798065662384
Overarching Label: Law, Score: 0.9606895446777344


Processing Topics: 100%|██████████| 2000/2000 [2:18:51<00:00,  4.17s/it]

Label: Bible, Score: 0.9975841641426086
Label: Interpretation, Score: 0.9958873987197876
Label: Translation, Score: 0.9937835335731506
Label: Septuagint, Score: 0.9902187585830688
Label: Concordance, Score: 0.9356622099876404
Label: Manuscripts, Score: 0.9228111505508423
Label: Hebrew, Score: 0.8729898929595947
Label: Tradition, Score: 0.8612541556358337
Overarching Label: Religion, Score: 0.9984269142150879
Overarching Label: Culture, Score: 0.951654314994812
Overarching Label: Scholarship, Score: 0.9105439186096191
Overarching Label: Literature, Score: 0.8734690546989441
Overarching Label: Language, Score: 0.8168816566467285


### Same as above but model runs locally

In [ ]:
# Only run once
from transformers import pipeline
from transformers import AutoTokenizer

hypothesis_template = "This text is about: {}"
tokenizer = AutoTokenizer.from_pretrained("MoritzLaurer/deberta-v3-large-zeroshot-v2.0")
zeroshot_model = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0")

for i, topics in tqdm(enumerate(top_k_topics), total=len(top_k_topics), desc="Processing Topics"):
    topic_labels_list = []
    overarching_labels_list = []
    for topic_index in topics:
        # Using themes from the themes_dict
        labels = themes_dict[str(topic_index)]
        labels = labels.split(", ")
        topic_labels_list.append(labels)
        
        # Using overarching themes from the overarching_themes_dict
        overarching_labels = overarching_themes_dict[str(topic_index)]
        overarching_labels = overarching_labels.split(", ")
        overarching_labels_list.append(overarching_labels)
    
    # Flatten the list of lists and remove duplicates
    topic_list = [item for sublist in topic_labels_list for item in sublist]
    topic_list = list(set(topic_list))
    overarching_list = [item for sublist in overarching_labels_list for item in sublist]
    overarching_list = list(set(overarching_list))
    
    # Apply the zeroshot model
    output = zeroshot_model(data[i], topic_list, hypothesis_template=hypothesis_template, multi_label=True)
    append_to_json('results.json', output)
    
    output_overarching = zeroshot_model(data[i], overarching_list, hypothesis_template=hypothesis_template, multi_label=True)
    append_to_json('overarching_results.json', output_overarching)
    
    # Print labels with scores > 0.7
    for label, score in zip(output['labels'], output['scores']):
        if score > 0.7:
            print(f"Label: {label}, Score: {score}")
            
    for label, score in zip(output_overarching['labels'], output_overarching['scores']):
        if score > 0.5:
            print(f"Overarching Label: {label}, Score: {score}")